In [1]:
import $ivy.`io.monix::monix:3.0.0`, monix._
import monix.eval.Task
import monix.reactive.Observable
import scala.concurrent.duration._
import monix.execution.Scheduler.Implicits.global

def getRecordsTask1(shardIt: String): Task[(List[String], Option[String])] = {
    val records = (1 to 2).map { i => s"[shard-it: $shardIt record: $i]" }.toList
    if (shardIt.toInt < 3) {  
        Task((records, Some((shardIt.toInt + 1).toString)))
    } else {
        Task((records, None))
    }
}

val initialShardIterator = "0"
val allRecordsObs1 = Observable.interval(0.seconds)
 .scanTask(getRecordsTask1(initialShardIterator)) {
    case ((records, nextShardIt), _) =>
        nextShardIt match {
              case None => Task.pure((records, None))
              case Some(nextShardIt) => {
                getRecordsTask1(nextShardIt)
              }
        }
    }
    .takeWhileInclusive {
        case (_, nextShardIt) => nextShardIt.isDefined
    }
    .flatMap {
        case (records, _) => Observable.fromIterable(records)
    }
allRecordsObs1.toListL.runSyncUnsafe(1.second).foreach(println)

[shard-it: 1 record: 1]
[shard-it: 1 record: 2]
[shard-it: 2 record: 1]
[shard-it: 2 record: 2]
[shard-it: 3 record: 1]
[shard-it: 3 record: 2]


import $ivy.$                      , monix._

import monix.eval.Task

import monix.reactive.Observable

import scala.concurrent.duration._

import monix.execution.Scheduler.Implicits.global


defined function getRecordsTask1
initialShardIterator: String = "0"
allRecordsObs1: monix.reactive.Observable[String] = monix.reactive.internal.operators.ConcatMapObservable@75e4ec22

In [2]:
def getRecordsTask2(shardIt: String, acc: List[String]): Task[(List[String], Option[String])] = {
    val records = (1 to 2).map { i => s"[shard-it: $shardIt record: $i]" }.toList
    if (shardIt.toInt < 3) {  
        Task((acc ++ records, Some((shardIt.toInt + 1).toString)))
    } else {
        Task((acc ++ records, None))
    }
}

val allRecordsObs2 = Observable.interval(0.seconds)
 .scanTask(getRecordsTask2(initialShardIterator, acc = List.empty)) {
    case ((records, nextShardIt), _) =>
        nextShardIt match {
              case None => Task.pure((records, None))
              case Some(nextShardIt) => {
                getRecordsTask2(nextShardIt, records)
              }
        }
    }
    .takeWhileInclusive {
        case (_, nextShardIt) => nextShardIt.isDefined
    }
    .flatMap {
        case (records, _) => Observable.fromIterable(records)
    }

allRecordsObs2.toListL.runSyncUnsafe(1.second).foreach(println)

[shard-it: 0 record: 1]
[shard-it: 0 record: 2]
[shard-it: 1 record: 1]
[shard-it: 1 record: 2]
[shard-it: 0 record: 1]
[shard-it: 0 record: 2]
[shard-it: 1 record: 1]
[shard-it: 1 record: 2]
[shard-it: 2 record: 1]
[shard-it: 2 record: 2]
[shard-it: 0 record: 1]
[shard-it: 0 record: 2]
[shard-it: 1 record: 1]
[shard-it: 1 record: 2]
[shard-it: 2 record: 1]
[shard-it: 2 record: 2]
[shard-it: 3 record: 1]
[shard-it: 3 record: 2]


defined function getRecordsTask2
allRecordsObs2: Observable[String] = monix.reactive.internal.operators.ConcatMapObservable@580ea8